In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import datetime
from nsepython import *
import concurrent.futures
import time

In [2]:
# sentiment analysis libraries
import nltk
nltk.downloader.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/shubham/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
https://www.niftyindices.com/IndexConstituent/ind_nifty50list.csv

In [5]:
csv = pd.read_html('https://in.investing.com/indices/s-p-cnx-nifty-components')


HTTPError: HTTP Error 403: Forbidden

In [4]:
#NIFTY URLS
nifty_500_ticker_url = 'https://www.niftyindices.com/IndexConstituent/ind_nifty500list.csv'
nifty_500 = pd.read_csv(nifty_500_ticker_url)
nifty_500.to_csv('./datasets/NIFTY_500.csv')
nifty_200_ticker_url = 'https://www.niftyindices.com/IndexConstituent/ind_nifty200list.csv'
nifty_200 = pd.read_csv(nifty_200_ticker_url)
nifty_200.to_csv('./datasets/NIFTY_200.csv')
nifty_100_ticker_url = 'https://www.niftyindices.com/IndexConstituent/ind_nifty100list.csv'
nifty_100 = pd.read_csv(nifty_100_ticker_url)
nifty_100.to_csv('./datasets/NIFTY_100.csv')
nifty_50_ticker_url = 'https://www.niftyindices.com/IndexConstituent/ind_nifty50list.csv'
nifty_50 = pd.read_csv(nifty_50_ticker_url)
nifty_50.to_csv('./datasets/NIFTY_50.csv')

KeyboardInterrupt: 

In [ ]:
# Set universe
universe = nifty_50

In [ ]:
# Read CSV & create a tickers df
tickers_df = universe[['Symbol', 'Company Name']]
tickers_list = tickers_df['Symbol']

In [ ]:
# News URL
news_url = 'https://ticker.finology.in/company/'

In [ ]:
#fetch all available data
article_data = pd.read_csv('./NIFTY_500_Articles.csv', index_col=0)

In [ ]:
'''
# list to store article data
article_data = []
# list to store tickers for which data is unavailable
unavailable_tickers = []
# length of companies
companies_len = len(tickers_list)
tickers_length = companies_len
#days_limit = datetime.datetime.now() - datetime.timedelta(days=30) #only 30 days old or newer articles
print('Fetching Article data..')
for i,ticker in enumerate(tickers_list):
    print(i, ticker)
    url= '{}/{}'.format(news_url, ticker)
    header={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'}
    response = requests.get(url, headers=header)
    html = BeautifulSoup(response.content, 'lxml')
    news_links = html.select('#newsarticles > a')
    if len(news_links) == 0:
        print('No news found for {}'.format(ticker))
        unavailable_tickers.append(ticker)
        continue
    # for tickers which are not recognized by finology website, it returns home-page. There's also a news section on homepage. so for unrecognized tickers, this function will scrape general financial news instead of ticker specific news
    # var to store article count
    ticker_articles_counter = 0
    for link in news_links:
        art_title = link.find('span', class_='h6').text
        #separate date and time from datetime object
        date_time_obj = datetime.strptime(link.find('small').text, '%d %b %Y, %I:%M%p')
        #if (date_time_obj <= days_limit):
        #     continue
        art_date = date_time_obj.date().strftime('%Y/%m/%d')
        art_time = date_time_obj.time().strftime('%H:%M')
        article_data.append([ticker, art_title, art_date, art_time])
        ticker_articles_counter += 1
    print('No of articles for {} is {}'.format(ticker, ticker_articles_counter))
    if(ticker_articles_counter==0):
        print('{} added to unavailable list')
        unavailable_tickers.append(ticker)  
'''

In [ ]:
unavailable_tickers

In [ ]:
# run the loop again, but for the tickers for which news could not be fetched.


In [ ]:
#create a df from the news articles list
articles_df = pd.DataFrame(article_data, columns=['Ticker', 'Headline', 'Date', 'Time'])
articles_df

In [ ]:
# get list of tickers for which news was fetched
tickers_list = articles_df['Ticker'].unique()
tickers_list

In [ ]:
#update original tickers_df to remove tickers for which news was not available
tickers_df = tickers_df[tickers_df['Symbol'].isin(tickers_list)].reset_index(drop=True)

In [ ]:
articles_df

In [ ]:
article_data

In [ ]:
article_data.drop(['Unnamed: 0'], axis=1)

In [ ]:
len(article_data['Ticker'].unique())

In [ ]:
# sentiment analysis libraries
import nltk
nltk.downloader.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
vader = SentimentIntensityAnalyzer()

In [ ]:
art_scores_df = pd.DataFrame(articles_df['Headline'].apply(vader.polarity_scores).to_list())

In [ ]:
art_scores_df

In [ ]:
# create df with article details and scores merged.
art_scores_df = pd.merge(articles_df, art_scores_df, left_index=True, right_index=True)

In [ ]:
art_scores_df

In [ ]:
#art_scores_df.to_csv('NIFTY_500_Articles.csv')

In [ ]:
article_data

In [ ]:
ticker_metadata = pd.read_csv('./ticker_metadata.csv', index_col=0)

In [ ]:
'''
# Get Company,Sector,Industry,mCap Data
ticker_meta = []
new_length = len(tickers_list)
#new_length = 10
print('Fetching ticker meta data')
# whether to get only data available in tickers_list or the whole universe bc even if we take whole universe, we will later merge the dfs ticker_meta and art_scores_df
# but since we're planning on not to fetch industry and sector data that often, its better to take the universe meta

for i, ticker in enumerate(tickers_list):
    meta = nse_eq(ticker)
    print(i, ticker)
    try:
        sector = meta['industryInfo']['macro']
    except KeyError:
        print('{} sector info is not available'.format(ticker))
        sector = np.nan
        industry = np.nan
        mCap = np.nan
        companyName = np.nan
        ticker_meta.append([ticker, sector, industry, mCap, companyName])
        continue
    try:
        industry = meta['industryInfo']['industry']
    except KeyError:
        print('{} industry info is not available'.format(ticker))
        industry = np.nan
        mCap = np.nan
        companyName = np.nan
        ticker_meta.append([ticker, sector, industry, mCap, companyName])
        continue
    try:
        mCap = round((meta['priceInfo']['previousClose'] * meta['securityInfo']['issuedSize'])/1000000000, 2)
    except KeyError:
        print('{} mCap data is not available'.format(ticker))
        mCap = np.nan
        companyName = np.nan
        ticker_meta.append([ticker, sector, industry, mCap, companyName])
        continue
    try:
        companyName = meta['info']['companyName']
    except KeyError:
        print('{} company Name is not available'.format(ticker))
        companyName = np.nan
        ticker_meta.append([ticker, sector, industry, mCap, companyName])
    ticker_meta.append([ticker, sector, industry, mCap, companyName])

'''

In [ ]:
xc_indices = pd.read_csv('XC-tickers.csv')
xc_indices.columns = ['Ticker', 'XC-Sector', 'XC-Industry']

In [ ]:
xc_indices

In [ ]:
ticker_meta_df = pd.DataFrame(ticker_meta, columns=['Ticker', 'Sector', 'Industry', 'Market Cap', 'Company Name'])

In [ ]:
ticker_meta_df

In [ ]:
ticker_metadata = pd.merge(ticker_meta_df, xc_indices, on='Ticker')

In [ ]:
ticker_metadata

In [ ]:
#ticker_metadata.to_csv('ticker_matadata.csv')

In [ ]:
ticker_metadata.drop(['Unnamed: 0'], axis=1)

In [ ]:
article_data = article_data.drop(['Unnamed: 0'], axis=1)
ticker_metadata = ticker_metadata.drop(['Unnamed: 0'], axis=1)

In [ ]:
article_score_summary = article_data.groupby('Ticker').mean()

In [ ]:
article_score_summary

In [ ]:
pd.merge(article_score_summary, ticker_metadata,  on='Ticker', how='inner')

In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
ticker_scores = article_data.groupby('Ticker').mean().reset_index()

In [ ]:
ticker_scores

In [ ]:
final_df = pd.merge(ticker_metadata, ticker_scores, on='Ticker', how='inner')

In [ ]:
final_df.columns = ['Symbol','Macro-Sector','Industry', 'Market Cap (Billion Rs)','Company Name','XC-Sector', 'XC-Industry', 'Negative', 'Neutral', 'Positive', 'Sentiment Score']

In [ ]:
final_df

In [ ]:
import matplotlib.pyplot as plt
import plotly
import plotly.express as px

In [ ]:
print('Generating Plots')
fig = px.treemap(
    final_df, path=[px.Constant('Nifty 500'), 'XC-Sector', 'XC-Industry', 'Symbol'], values='Market Cap (Billion Rs)', color='Sentiment Score',
    hover_data=['Company Name', 'Negative', 'Neutral', 'Positive', 'Sentiment Score'], color_continuous_scale=['#FF0000', "#000000", '#00FF00'], color_continuous_midpoint=0
    )
fig.data[0].customdata = final_df[['Company Name', 'Negative', 'Neutral', 'Positive', 'Sentiment Score']]
fig.data[0].texttemplate = "%{label}<br>%{customdata[4]}"
fig.update_traces(textposition="middle center")
fig.update_layout(margin = dict(t=30, l=10, r=10, b=10), font_size=20)

--------------------------

In [ ]:
test_tickers = ['3MINDIA', 'APLAPOLLO', 'ABBOTINDIA', 'ABSLAMC', 'ALKYLAMINE', 'AMARAJABAT', 'AMBER', 'APOLLOTYRE', 'APTUS', 'ASTRAZEN', 'BEML', 'BAJAJELEC', 'BALKRISIND', 'BATAINDIA', 'BAYERCROP', 'BERGEPAINT', 'BHARATRAS', 'BLUEDART', 'BRITANNIA', 'MAPMYINDIA', 'CESC', 'CAMPUS', 'CEATLTD', 'CDSL', 'CENTURYPLY', 'CHOLAHLDNG', 'COLPAL', 'CUMMINSIND', 'DEVYANI', 'DIVISLAB', 'DIXON', 'EIHOTEL', 'EPL', 'EMAMILTD', 'NYKAA', 'FACT', 'FINEORG', 'GAIL', 'GICRE', 'GODFRYPHLP', 'GUJALKALI', 'GUJGASLTD', 'GSPL', 'HLEGLAS', 'HIKAL', 'HGS', 'HAL', 'HONAUT', 'HUDCO', 'IFBIND', 'INDIACEM', 'IBULHSGFIN', 'IRFC', 'IGL', 'IPCALAB', 'JBCHEPHARM', 'KNRCON', 'KPITTECH', 'KANSAINER', 'L&TFH', 'LAXMIMACH', 'LUXIND', 'MRF', 'MGL', 'M&MFIN', 'M&M', 'MAHLOG', 'MAZDOCK', 'MEDPLUS', 'METROPOLIS', 'MSUMI', 'MUTHOOTFIN', 'NLCINDIA', 'NATIONALUM', 'NAVINFLUOR', 'NESTLEIND', 'NAM-INDIA', 'NUVOCO', 'OBEROIRLTY', 'OIL', 'OLECTRA', 'PCBL', 'PATANJALI', 'PFIZER', 'POLYPLEX', 'PFC', 'PRESTIGE', 'PRINCEPIPE', 'PGHL', 'RAIN', 'RAINBOW', 'RATNAMANI', 'REDINGTON', 'RBA', 'SIS', 'SRF', 'SHILPAMED', 'SAIL', 'SUMICHEM', 'SUNTV', 'SUNDARMFIN', 'SUPREMEIND', 'SYMPHONY', 'TCNSBRANDS', 'TTML', 'TEJASNET', 'THERMAX', 'THYROCARE', 'TORNTPHARM', 'TIINDIA', 'UPL', 'UTIAMC', 'VIPIND', 'VAIBHAVGBL', 'VBL', 'MANYAVAR', 'VIJAYA', 'VOLTAS', 'WHIRLPOOL', 'WOCKPHARMA', 'ZFCVINDIA', 'ZEEL', 'ZENSARTECH', 'ZOMATO', 'ECLERX']

In [ ]:
len(test_tickers)

In [ ]:
news_url = 'https://ticker.finology.in/company/'
# list to store article data
article_data = []
unavailable_tickers = []
# list to store tickers for which data is unavailable
# length of companies
companies_len = len(test_tickers)
tickers_length = companies_len
#days_limit = datetime.datetime.now() - datetime.timedelta(days=30) #only 30 days old or newer articles
print('Fetching Article data..')
for i in range(tickers_length):
    print(i, test_tickers[i])
    url= '{}/{}'.format(news_url, test_tickers[i])
    header={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'}
    response = requests.get(url, headers=header)
    html = BeautifulSoup(response.content, 'lxml')
    news_links = html.select('#newsarticles > a')
    if len(news_links) == 0:
        print('No news found for {}'.format(test_tickers[i]))
        unavailable_tickers.append(test_tickers[i])
        continue
    # for tickers which are not recognized by finology website, it returns home-page. There's also a news section on homepage. so for unrecognized tickers, this function will scrape general financial news instead of ticker specific news
    # var to store article count
    ticker_articles_counter = 0
    for link in news_links:
        art_title = link.find('span', class_='h6').text
        #separate date and time from datetime object
        date_time_obj = datetime.datetime.strptime(link.find('small').text, '%d %b %Y, %I:%M%p')
        #if (date_time_obj <= days_limit):
        #     continue
        art_date = date_time_obj.date().strftime('%Y/%m/%d')
        art_time = date_time_obj.time().strftime('%H:%M')
        article_data.append([test_tickers[i], art_title, art_date, art_time])
        ticker_articles_counter += 1
    if(ticker_articles_counter==0):
        print('Article length = 0 for {}'.format(test_tickers[i]))
        unavailable_tickers.append(tickers_list[i])  

In [ ]:
pprint(article_data)

--------------------------

In [ ]:
url= 'https://ticker.finology.in/company/EMAMILTD'
header={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'}
response = requests.get(url, headers=header)
html = BeautifulSoup(response.content, 'lxml')

In [ ]:
news_links = html.select('#newsarticles > a')

In [ ]:
news_links